In [60]:
import pandas as pd 
import numpy as np
import quandl
import datetime
import math



In [74]:
recap = pd.DataFrame(index =["Valeur absolue","Distance Euclidienne","Score3"],columns=['GARCH', 'ARIMA', 'LSTM', 'ML2', 'ML3']).fillna(0)
recap

,GARCH,ARIMA,LSTM,ML2,ML3
Valeur absolue,0,0,0,0,0
Distance Euclidienne,0,0,0,0,0
Score3,0,0,0,0,0


In [142]:
companies=['AAPL','ABT','ACN','ATVI','ADBE','AMD','GOOGL','GOOG','AMZN','AAL','BLK','CBS'] # 'BA' fait n'imp
#companies

In [143]:
L_abs = []
L_abs_per = []
L_eucli = []
L_eucli_per =[]


for l in companies: 
    path=r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Code\lstm_16_17_"+l+".csv" #Ordi Fixe Geoffroy
    #path=r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Code\lstm_16_17_"+l+".csv" #Ordi Portable Geoffroy
    df=pd.read_csv(path) 
    #df=pd.read_csv("C:\Users\Geoffroy\Desktop\IT\Projet Supélec\SP500.xlsx") #PC Portable Geoffroy
    V_abs = df['Ecart'].mean()
    V_abs_perc = V_abs/(df['True Close'].tolist()[-1])
    V_eucli = ((df['Ecart']**2).mean())**(1/2)
    V_eucli_per =V_eucli/(df['True Close'].tolist()[-1])
    L_abs.append([l ,V_abs])
    L_abs_per.append([l,V_abs_perc])
    L_eucli.append([l,V_eucli])
    L_eucli_per.append([l,V_eucli_per])


                          


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 39: invalid continuation byte

In [131]:
df_absp= pd.DataFrame(L_abs_per)
df_ep = pd.DataFrame(L_eucli_per)

df_absp.columns=['Entreprise','Valeur Absolue']
df_ep.columns=['Entreprise','Moindres Carrées']

df_absp=df_absp.set_index('Entreprise')
df_ep=df_ep.set_index('Entreprise')

dfscore=df_absp.join(df_ep)
dfscore.style.format("{:.2%}")

#Score 3 cross entrop y ? probleme car nos valeurs sont pas entre 0 et 1...




,Valeur Absolue,Moindres Carrées
Entreprise,,
AAPL,6.41%,8.01%
ABT,88.37%,151.55%
ACN,2.49%,2.71%
ATVI,5.27%,6.29%
ADBE,6.98%,7.98%
AMD,31.61%,33.43%
GOOGL,1.86%,2.10%
GOOG,3.16%,4.37%
AMZN,2.02%,2.35%


In [141]:
VA=dfscore['Valeur Absolue'].mean()
VMC=dfscore['Moindres Carrées'].mean()
Score3 = 0
recap['LSTM']=[VA,VMC,Score3]

recap.style.format("{:.2%}")



,GARCH,ARIMA,LSTM,ML2,ML3
Valeur absolue,0.00%,0.00%,14.65%,0.00%,0.00%
Distance Euclidienne,0.00%,0.00%,21.15%,0.00%,0.00%
Score3,0.00%,0.00%,0.00%,0.00%,0.00%


# Données Financières

In [140]:
S = pd.read_excel(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data\Entreprises_S&P.xlsx", sheetname='Feuil1')
logo=list(S['Logo'])[:3] #3 mi ici pour une question de temps
start = datetime.datetime(2005,1,1)
end = datetime.date.today()

D:\04.Programme\Anaconda\envs\tensorflow_env\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Geoffroy\\Desktop\\IT\\Projet Supélec\\Data\\Entreprises_S&P.xlsx'

In [125]:
data_fi=pd.DataFrame({logo[0] : quandl.get("WIKI/" +logo[0] , start_date=start, end_date=end)['Adj. Close']})

for l in logo[1:]:
    data=pd.DataFrame({l : quandl.get("WIKI/" +l , start_date=start, end_date=end)['Adj. Close']})
    data_fi=data_fi.join(data)
    

#data_fi.to_excel('Data Financière.xlsx')
#data_fi.to_csv('Data Financière.csv')


# Données Simmulées

# Forecast

In [22]:
#Que les méthode de forecast enregistre le tout dans un csv
# ex :
# garch = forecast_garch(L[0])
# for l in logo[1:]:
#     temp= forecast_garch(l)
#     garch=garch.join(temp)

# garch.to_csv('GARCH.CSV')

# De même avec les autres méthode


In [ ]:
# garch = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\GARCH.csv)
# arima = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ARIMA.csv)
# ml1 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML1.csv)                       
# ml2 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML2.csv)
# ml3 = pandas.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\ML3.csv)

# Détermination du score pour Data Financière

###  Valeur Absolue

#### GARCH

In [ ]:
list_abs=[]
for l in logo:
    ecart = abs(data_fi[l]-garch[l]).dropna().mean()
    list_abs.append(ecart)
    
Score_abs = mean(list_abs)

recap['GARCH']['Valeur absolue']= score_abs

# Y à t'il un moyen de créer une méthode auto(score,Méthode de forecast,type de data) pour pas avoir à le refaire pour tous les différents score et méthodes et type de data (simmulé ou réelle)

